# TASK 2 : Restaurant Recomendation

In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [10]:
df = pd.read_csv(r"C:\Users\Omkar Waghmare\OneDrive\Desktop\WORK\COGNIFYZ intern\Dataset .csv")

In [11]:
df.sample(3)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
1891,18441701,Wow! Momo,1,Gurgaon,"Shop 214, Huda Market, Sector 31, Gurgaon",Sector 31,"Sector 31, Gurgaon",77.050958,28.453489,"Chinese, Tibetan",...,Indian Rupees(Rs.),No,No,No,No,1,2.9,Orange,Average,9
7458,18244564,Foodaholic,1,New Delhi,"ED 118, Tagore Garden, New Delhi",Tagore Garden,"Tagore Garden, New Delhi",77.115140,28.648811,"Chinese, North Indian",...,Indian Rupees(Rs.),No,No,No,No,1,0.0,White,Not rated,1
8196,18382347,My Corn,1,Noida,"C-28/29, Food Court, Tower B, Logix Cyber Park...","Logix Cyber Park, Sector 62, Noida","Logix Cyber Park, Sector 62, Noida, Noida",77.366672,28.612740,Fast Food,...,Indian Rupees(Rs.),No,No,No,No,1,0.0,White,Not rated,0


**Preprocessing data**

In [12]:
df['Cuisines'].fillna('Not Specified', inplace=True)

label_encoder = LabelEncoder()
categorical_columns = ['City', 'Locality', 'Locality Verbose', 'Cuisines', 'Currency', 'Rating color', 'Rating text']

for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])
print(df.head())

   Restaurant ID         Restaurant Name  Country Code  City  \
0        6317637        Le Petit Souffle           162    73   
1        6304287        Izakaya Kikufuji           162    73   
2        6300002  Heat - Edsa Shangri-La           162    75   
3        6318506                    Ooma           162    75   
4        6314302             Sambo Kojin           162    75   

                                             Address  Locality  \
0  Third Floor, Century City Mall, Kalayaan Avenu...       171   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...       593   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...       308   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...       862   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...       862   

   Locality Verbose   Longitude   Latitude  Cuisines  ...  Currency  \
0               172  121.027535  14.565443       920  ...         0   
1               601  121.014101  14.553708      1111  ...         0   
2    

**Determining criteria for restaurant recommendation**

In [13]:
user_preferences = {
    'Cuisine Preference': 'Italian',  
    'Price Range': 2, 
    'Minimum Rating': 4.0 
}

filtered_restaurants = df[
    (df['Cuisines'].astype(str).str.contains(user_preferences['Cuisine Preference'], case=False)) &
    (df['Price range'] <= user_preferences['Price Range']) &
    (df['Aggregate rating'] >= user_preferences['Minimum Rating'])
]

print("Recommended Restaurants:")
print(filtered_restaurants[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating']].head())


Recommended Restaurants:
Empty DataFrame
Columns: [Restaurant Name, Cuisines, Price range, Aggregate rating]
Index: []


In [14]:
user_preferences_adjusted = {
    'Price Range': 2, 
    'Minimum Rating': 4.0  
}

filtered_restaurants_adjusted = df[
    (df['Price range'] <= user_preferences_adjusted['Price Range']) &
    (df['Aggregate rating'] >= user_preferences_adjusted['Minimum Rating'])
]

print("Recommended Restaurants with Adjusted Preferences:")
print(filtered_restaurants_adjusted[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating']].head())


Recommended Restaurants with Adjusted Preferences:
                Restaurant Name  Cuisines  Price range  Aggregate rating
18   Hobing Korean Dessert Cafe       452            2               4.5
56                   Leme Light       257            2               4.2
104       Mama's Boy Restaurant      1739            1               4.5
105                   Sr. Sol 1      1212            1               4.6
108         Big City Bread Cafe       282            1               4.3


**content-based filtering approach**

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

df['Cuisines'].fillna('Not Specified', inplace=True)

df['Price range'] = df['Price range'].astype(str)
df['Aggregate rating'] = df['Aggregate rating'].astype(str)

df['Features'] = df.apply(lambda row: f"{row['Cuisines']} {row['Price range']} {row['Aggregate rating']}", axis=1)

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Features'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

def recommend_restaurants(user_preferences, cosine_similarities, df):
    query = f"{user_preferences['Cuisine Preference']} {user_preferences['Price Range']} {user_preferences['Minimum Rating']}"

    query_vector = tfidf_vectorizer.transform([query])

    cosine_similarities_query = linear_kernel(query_vector, tfidf_matrix).flatten()

    restaurant_indices = cosine_similarities_query.argsort()[::-1]

    recommended_indices = restaurant_indices[1:]

    top_recommendations = df.iloc[recommended_indices].head(user_preferences['Top N Recommendations'])
    
    return top_recommendations[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating']]

user_preferences_content_based = {
    'Cuisine Preference': 'Italian',
    'Price Range': '2',
    'Minimum Rating': '4.0',
    'Top N Recommendations': 5
}

content_based_recommendations = recommend_restaurants(user_preferences_content_based, cosine_similarities, df)

print("Content-Based Recommendations:")
print(content_based_recommendations)


Content-Based Recommendations:
                                Restaurant Name  Cuisines Price range  \
3188  Cake Central - Premier Cake Design Studio       186           1   
3186                                    Barista       331           2   
3185                          Anil Mishtan Wala      1262           1   
3184                                   Aka Saka       533           3   
3183                             RR China Tawun       497           1   

     Aggregate rating  
3188              3.4  
3186              3.3  
3185              3.1  
3184              3.3  
3183              0.0  


**Test the recommendation system**

In [16]:
sample_user_preferences = {
    'Cuisine Preference': 'Chinese',
    'Price Range': '3',
    'Minimum Rating': '4.5',
    'Top N Recommendations': 5
}

sample_recommendations = recommend_restaurants(sample_user_preferences, cosine_similarities, df)

print("Sample Recommendations:")
print(sample_recommendations)


Sample Recommendations:
                                Restaurant Name  Cuisines Price range  \
3188  Cake Central - Premier Cake Design Studio       186           1   
3186                                    Barista       331           2   
3185                          Anil Mishtan Wala      1262           1   
3184                                   Aka Saka       533           3   
3183                             RR China Tawun       497           1   

     Aggregate rating  
3188              3.4  
3186              3.3  
3185              3.1  
3184              3.3  
3183              0.0  
